In [1]:
import codecs
import ctypes
import os
import subprocess
import pandas as pd

In [2]:
from tqdm import tqdm

In [3]:
import codecs
import time
import logging, logging.config
from datetime import datetime, timedelta

In [4]:
import subprocess
import time
import win32com.client
import pandas as pd

In [5]:
import numpy as np
import pandas as pd

In [6]:
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
objStockOrder = win32com.client.Dispatch("CpTrade.CpTd0311")
objTrade =  win32com.client.Dispatch("CpTrade.CpTdUtil")
objRq = win32com.client.Dispatch("CpTrade.CpTd6033")
objTdNew = win32com.client.Dispatch('CpTrade.CpTdNew5331A') 

In [7]:
objSeries = win32com.client.Dispatch("CpIndexes.CpSeries")
objIndex = win32com.client.Dispatch("CpIndexes.CpIndex")

In [8]:
"""
0 - 가격지표
1 - 대신개발지표
2 - 추세지표
3 - 변동성지표
4 - 거래량지표
5 - 기간활용지표
6 - 투자자별지표
"""

'\n0 - 가격지표\n1 - 대신개발지표\n2 - 추세지표\n3 - 변동성지표\n4 - 거래량지표\n5 - 기간활용지표\n6 - 투자자별지표\n'

In [9]:
# objIndex.GetChartIndexCodeListByIndex(3)

In [10]:
IndexList = [
#     '이격도',
    'AB Ratio',
#     'ADX',
#     'ADXR',
    'Aroon',
    'Aroon Osillator',
    'ATR',
    'BB-RSI',
    'BPDL RSI',
    'BPDL Stochatic',
    'Chande Momentum Oscillator',
    "Chaikin's Volatility",
    'CompuTrac Volatility',
    'DMI',
    'DMI(simple)',
    'Energy',
    'High Low Envelope',
    'High Low Oscillator',
    'Inertia',
    'Klinger Oscillator',
    'Linear Trend Oscillator',
    'LRS',
    'Mass Index',
    'Morris Mixed Momentum',
    'New BPDL HiLo Index MA',
    'Open Difference',
    'QStick',
    'Random Walk Index',
    'RCI',
    'Relative Volatility Index',
    'Reverse',
    'RSI',
    'RSI(simple)',
#     'RSI(stochastic)',
    'R-Squared',
    'Sigma',
    'SMI',
    'Standard Deviation',
    'Standard Error',
    'Stochastic Fast',
    'Stochastic Slow',
    'Stochastic(simple)',
    'Stochastic Oscillator',
    'TRIX기울기',
    'True Range',
    'TSF Oscillator',
    'VHF'
]

In [11]:
data = pd.read_table("/Users/jooh8/Documents/GitHub/Quant/data/A005930/DAY_A005930.txt", delimiter=" ")

In [12]:
tolist = data["종가 시가 고가 저가 거래량".split(" ")].values.tolist()

In [13]:
print(len(tolist))

10592


In [14]:
for close, open, high, low, vol in tolist:

    objSeries.Add(close, open, high, low, vol)
    
objIndex.series = objSeries

In [15]:
for INDEXNAME in tqdm(IndexList):
    
    print(INDEXNAME)
    print("#"*80)
    
    objIndex.put_IndexKind(INDEXNAME)     # 계산할 지표: MACD
    objIndex.put_IndexDefault(INDEXNAME)  # MACD 지표 기본 변수 자동 세팅

    indexID = objIndex.GetChartIndexByName(INDEXNAME)
    indexName = objIndex.GetLineResults(indexID)

    # print(indexID)
    # print(indexName)
    # print(objIndex.get_Term1(), objIndex.get_Term2(), objIndex.get_Signal())

    # 지표 데이터 계산 하기
    objIndex.Calculate()
    cntofIndex = objIndex.ItemCount

    # print(cntofIndex)

    creonIndex = {}
    for index in range(cntofIndex):
        cnt = objIndex.GetCount(index)
        creonIndex[indexName[index]] = []
        for j in range(cnt) :
            value = objIndex.GetResult(index, j)
            creonIndex[indexName[index]].append(value)

    for k, v in creonIndex.items():
        data[k] = v

  0%|                                         | 0/43 [00:00<?, ?it/s]

AB Ratio
################################################################################


  2%|▊                                | 1/43 [00:00<00:05,  7.35it/s]

Aroon
################################################################################


  5%|█▌                               | 2/43 [00:00<00:05,  7.59it/s]

...

 93%|█████████████████████████████▊  | 40/43 [00:04<00:00,  8.56it/s]

True Range
################################################################################
TSF Oscillator
################################################################################


 98%|███████████████████████████████▎| 42/43 [00:04<00:00,  9.20it/s]

VHF
################################################################################


100%|████████████████████████████████| 43/43 [00:05<00:00,  8.81it/s]


In [18]:
data.columns

Index(['날짜', '시가', '고가', '저가', '종가', '거래량', '거래대금', '상장주식수', '시가총액',
       '외국인현보유수량', '기관순매수', 'A Ratio_20', 'B Ratio_20', 'Aroon_20 Up', 'Down',
       'Aroon Osillator_20', 'ATR_14', 'Signal_9', 'BB-RSI_종가_10',
       'BB-RSI_20_2.00 상한', 'BB-RSI 하한', 'BPDL RSI_14', 'BPDL Stochatic_14',
       'Chande Momentum Oscillator_10', 'Chaikin's Volatility_14,14',
       'CompuTrac Volatility_10', '+DI_14', '-DI_14', 'ADX_14', 'ADXR_14',
       '+DI(simple)_14', '-DI(simple)_14', 'ADX(simple)_14,14',
       'ADXR(simple)_14,14,14', 'Energy+_14', 'Energy-_14',
       'High Low Envelope_1', 'High Low Oscillator_3', 'Inertia_20,14,10',
       'Klinger Oscillator', 'Signal_13', 'Linear Trend Oscillator_10,20',
       'LRS_종가,14', 'Mass Index_20,9', 'Morris Mixed Momentum',
       'New BPDL HiLo Index MA_14,7', 'Open Difference_15', 'QStick_14',
       'Random Walk Index_15,3', 'RCI_5', 'RCI_9', 'RCI_13', 'RCI_18',
       'Relative Volatility Index_단순,14,10', 'Signal_단순,9', 'Reverse 단기_12',
    

In [17]:
data.tail()

,날짜,시가,고가,저가,종가,거래량,거래대금,상장주식수,시가총액,외국인현보유수량,...,Fast %D_3,"Slow %K_5,3",Slow %D_3,"Slow(Simple) %K_5,3",Slow(Simple) %D_3,"StochOsc_5,3","TRIX기울기_종가,14",True Range,"TSF Oscillator_종가,14,28",VHF_14
10587,20190507,45250,45300,44400,44850,12014907,538125000000,5969782000,267744722000000,3420515,...,55.894737,58.695652,74.154787,46.718866,58.417012,-32.210526,-0.010055,900.0,-648.076923,0.448276
10588,20190508,44300,44850,44200,44250,10398754,462087000000,5969782000,264162853000000,3420231,...,28.688388,30.769231,54.932739,24.549909,41.483461,-26.307435,-0.014113,650.0,-460.317460,0.451613
10589,20190509,43900,44250,42450,42450,23029718,993612000000,5969782000,253417245000000,3417828,...,8.688388,6.493506,31.986130,12.274955,26.879208,-8.688388,-0.025350,1800.0,-435.012210,0.474359
10590,20190510,42600,43450,42450,42900,14579512,624673000000,5969782000,256103671000000,3415182,...,4.960317,5.319149,14.193962,12.387477,19.633343,7.539683,-0.029019,1000.0,-451.892552,0.544118
10591,20190513,42500,43200,42350,42650,7635079,326874000000,5969782000,254611225000000,3414755,...,7.556497,7.317073,6.376576,11.278484,15.455913,2.612994,-0.031157,850.0,-513.858364,0.548148


In [16]:
data[data.columns[-5:]].tail()

,"StochOsc_5,3","TRIX기울기_종가,14",True Range,"TSF Oscillator_종가,14,28",VHF_14
10587,-32.210526,-0.010055,900.0,-648.076923,0.448276
10588,-26.307435,-0.014113,650.0,-460.317460,0.451613
10589,-8.688388,-0.025350,1800.0,-435.012210,0.474359
10590,7.539683,-0.029019,1000.0,-451.892552,0.544118
10591,2.612994,-0.031157,850.0,-513.858364,0.548148
